# MCP Server Deployment on AgentCore Runtime

## 1. Environment Setup

### 1.1 Install Required Dependencies

In [ ]:
!printf "✅ Python version: $(python3 --version)\n"

!python3 -m pip install --requirement requirements.txt --upgrade --quiet

!echo "✅ Requirements installed"

## 2. Authentication Setup

### 2.1 Create Cognito User Pool

In [ ]:
from aws_setup import setup_cognito_user_pool

print("Setting up Amazon Cognito user pool (checking for existing pools first)...")
try:
    cognito_config = setup_cognito_user_pool()
    if cognito_config is None:
        raise Exception("Failed to setup Cognito user pool")
    print("✅ Cognito setup completed")
except Exception as e:
    print(f"❌ Error setting up Cognito: {e}")
    raise

### 2.2 Verify Cognito Configuration

In [ ]:
print(f'✅ Username: {cognito_config["AuthParameters"]["USERNAME"]}')
print(f'✅ PASSWORD: ***MASKED***')
print(f'✅ Bearer Token: ***MASKED***')
print(f'✅ Client ID: {cognito_config["client_id"]}')
print(f'✅ Pool ID: {cognito_config["pool_id"]}')
print(f'✅ Discovery URL: {cognito_config["discovery_url"]}')

## 3. AgentCore Runtime Deployment

### 3.1 Initialize AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"✅ Using AWS region: {region}")

agentcore_runtime = Runtime()
print("✅ AgentCore Runtime initialized")

### 3.2 Check Required Files

In [ ]:
import os

required_files = [
    'awslabs/aws_support_mcp_server/server.py',
    'requirements.txt',
]

try:
    for file in required_files:
        print(f"Checking: {file}")

        if not os.path.exists(file):
            raise FileNotFoundError(f"Required file {file} not found")

    print("✅ All required files found")
except FileNotFoundError as e:
    print(f"❌ {e}")
    raise

### 3.3 Configure JWT Authentication

In [ ]:
if 'cognito_config' not in locals() or not cognito_config:
    raise Exception("Cognito configuration not available")

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print(f"✅ Auth config prepared for client: {cognito_config['client_id']}")

### 3.4 Create IAM Execution Role with "AWSSupportAccess" policy attached

In [ ]:
from aws_setup import get_or_create_role

from config import AGENT_NAME

print("Getting or creating IAM role...")
execution_role_arn = get_or_create_role(AGENT_NAME)
print(f"✅ Using role: {execution_role_arn}")

### 3.5 Deploy MCP Server

In [ ]:
import nest_asyncio

from config import AGENT_NAME

nest_asyncio.apply()

print("Configuring AgentCore Runtime...")
print(f"ℹ️ Agent name: {AGENT_NAME}")
print(f"ℹ️ Execution role: {execution_role_arn}")
print(f"ℹ️ Region: {region}")
response = agentcore_runtime.configure(
    entrypoint="awslabs/aws_support_mcp_server/server.py",
    execution_role=execution_role_arn,
    agent_name=AGENT_NAME,
    requirements_file="requirements.txt",
    container_runtime="auto",
    auto_create_ecr=True,
    auto_create_execution_role=False,
    authorizer_configuration=auth_config,
    region=region,
    protocol="MCP",
    disable_otel=False,
)
print("✅ Configuration completed")
print(f"Response: {response}")

### 3.6 Start Runtime

In [ ]:
print("Launching MCP server to AgentCore Runtime (This might take several minutes...)")
launch_result = agentcore_runtime.launch(
    # Explicitly defined local/local_build as False to ensure it would be built with cloud
    local=False,
    local_build=False,
    auto_update_on_conflict=True,
)
print("✅ Launch completed")

print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### 3.7 Check Runtime Status

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
max_wait_time = 1800  # 30 minutes
wait_time = 0

while status not in end_status and wait_time < max_wait_time:
    print(f"Status: {status} - waiting... ({wait_time}s/{max_wait_time}s)")
    time.sleep(10)
    wait_time += 10

    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if wait_time >= max_wait_time:
    print(f"⚠️ Timeout reached. Current status: {status}")
elif status == 'READY':
    print(f"✅ AgentCore Runtime is READY!")
else:
    print(f"⚠️ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## 4. Configuration Storage

### 4.1 Save Configuration to AWS

In [ ]:
import boto3
import json

from config import PARAMETER_NAME, SECRETS_NAME

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name=SECRETS_NAME,
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✅ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId=SECRETS_NAME,
        SecretString=json.dumps(cognito_config)
    )
    print("✅ Cognito credentials updated in Secrets Manager")
except Exception as e:
    print(f"❌ Error managing secrets: {e}")
    raise

try:
    agent_arn_response = ssm_client.put_parameter(
        Name=PARAMETER_NAME,
        Value=launch_result.agent_arn,
        Type='String',
        Description='Agent ARN for MCP server',
        Overwrite=True
    )
    print("✅ Agent ARN stored in Parameter Store")
except Exception as e:
    print(f"❌ Error storing Agent ARN: {e}")
    raise

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## ✅ Deployment Complete

The MCP Server has been successfully deployed on AgentCore Runtime.

## 5. Test & Validate

### 5.1 Test MCP Connection

In [ ]:
from mcp_client_test import test_mcp_connection

# Run the MCP client test
await test_mcp_connection()